In [ ]:

#https://www.pythonforengineers.com/audio-and-digital-signal-processingdsp-in-python/
import numpy as np
import wave
import struct
import matplotlib.pyplot as plt



In [ ]:
frequency = 1000 # number of timers a wave repeats a second
num_samples = 48000
sampling_rate = 48000.0 # sampling rate of the analog to digital convert

amplitude = 16000 # the half of the max value of short int which use in wav file
file = "/home/leliang/project/baidu/tmp/leliang/test.wav"

# 2 * pi * frequency  == 2 * pi / T ; x/sampling_rate == the time cost
sine_wave = [np.sin(2 * np.pi * frequency * x/sampling_rate) for x in range(num_samples)]

nframes=num_samples
comptype="NONE"
compname="not compressed"
nchannels=1
sampwidth=2 # short int

wav_file = wave.open(file, 'w')
#print(help(wav_file.setparams))
wav_file.setparams((nchannels, sampwidth, int(sampling_rate), nframes, comptype, compname))

for s in sine_wave:
    wav_file.writeframes(struct.pack('h', int(s*amplitude)))
    
wav_file.close()

In [ ]:
#try to get the frequency
frame_rate = 48000.0
infile = "/home/leliang/project/baidu/tmp/leliang/test.wav"
num_samples= 48000
wav_file = wave.open(infile, 'r')
data = wav_file.readframes(num_samples)
wav_file.close()

data = struct.unpack('{n}h'.format(n=num_samples), data)
data = np.array(data)

data_fft = np.fft.fft(data)


In [ ]:
frequencies = np.abs(data_fft)

#The FFT returns all possible frequencies in the signal. And the way it returns is that each index contains a frequency element. 
#Say you store the FFT results in an array called data_fft. Then:
#data_fft[1] will contain frequency part of 1 Hz.
#
#data_fft[1000] will contain frequency part of 1000 Hz.
#if you have no 1Hz frequency in your signal, it will be minuscule.
print("The frequency is {} Hz".format(np.argmax(frequencies)))
print("length of frequencies:", len(frequencies))

plt.subplot(2,1,1)
plt.subplots_adjust(hspace=0.5)
plt.plot(data[:300])
plt.title("Original audio wave")

plt.subplot(2,1,2)
plt.plot(frequencies)
plt.title("Frequencies found")
#plt.xlim(0,1200)
 
plt.show()

In [ ]:
np.abs(data_fft[:8])

In [ ]:
np.sqrt(6.2469663*6.2469663 + -11.9802755*-11.9802755)

In [ ]:
#cleaning a noisy sine wave

# frequency is the number of times a wave repeats a second
frequency = 1000
noisy_freq = 50
num_samples = 48000
# The sampling rate of the analog to digital convert
sampling_rate = 48000.0

#Create the sine wave and noise
sine_wave = [np.sin(2 * np.pi * frequency * x1 / sampling_rate) for x1 in range(num_samples)]
sine_noise = [np.sin(2 * np.pi * noisy_freq * x1/  sampling_rate) for x1 in range(num_samples)]
 
#Convert them to numpy arrays
sine_wave = np.array(sine_wave)
sine_noise = np.array(sine_noise)

# Add them to create a noisy signal
combined_signal = sine_wave + sine_noise

plt.subplot(3,1,1)
plt.title("Original sine wave")
# Need to add empty space, else everything looks scrunched up!
plt.subplots_adjust(hspace=1.5)
plt.plot(sine_wave[:500])
 
plt.subplot(3,1,2)
plt.title("Noisy wave")
plt.plot(sine_noise[:4000])
 
plt.subplot(3,1,3)
plt.title("Original + Noise")
plt.plot(combined_signal[:3000])

plt.show()


In [ ]:
data_fft = np.fft.fft(combined_signal)
freq = (np.abs(data_fft))

plt.plot(freq)
plt.title("Before filtering: Will have main signal (1000Hz) + noise frequency (50Hz)")
plt.xlim(0,1200)

In [ ]:
filtered_freq = [f if (950 < index < 1050 and f > 1) else 0 for index, f in enumerate(freq)]

plt.plot(filtered_freq)
plt.title("After filtering: Main signal only (1000Hz)")
plt.xlim(0,1200)
plt.show()
plt.close()

In [ ]:
recovered_signal = np.fft.ifft(filtered_freq)

plt.subplot(3,1,1)
plt.title("Original sine wave")
# Need to add empty space, else everything looks scrunched up!
plt.subplots_adjust(hspace=.8)

plt.plot(sine_wave[:500])

plt.subplot(3,1,2)
plt.title("Noisy wave")
plt.plot(combined_signal[:4000])
 
plt.subplot(3,1,3)
plt.title("Sine wave after clean up")
plt.plot((recovered_signal[:500]))

plt.show()

In [ ]:
help(wave.open)

帕斯瓦尔定理： 信号在时域和频域上的能量相等
$$\sum_{n=0}^{N-1}{|x[n]|^2} = \frac {1} {N} \sum_{k=0}^{N-1}{|X[k]|^2}$$

其中x为时域实数数据，X为对应的频域复数数据

In [ ]:
#帕斯瓦尔定理
#help(np.random.randint)
samples_size = 1000
test_array = np.random.randint(1, 10000, samples_size)
energy_value = np.sum(test_array**2)
print("energy_value:", energy_value)

energy_value2 = np.sum(np.abs(np.fft.fft(test_array))**2)/samples_size
print("energy_value2:", energy_value2)

经过FFT后有N个元素的序列x经过傅里叶变化生成有N个元素的y
$$ y[k] = \sum_{n=0}^{N-1}{e^{-2\pi j \frac {kn}{N}}}{x[n]}$$
反FFT
$$ x[n] = \frac {1} {N} \sum_{k=0}^{N-1}{e^{2\pi j \frac {kn}{N}}}{y[k]}$$

欧拉公式：
$$e^{\theta j} = cos(\theta) + jsin(\theta)$$

经过FFT后变为复数形式：$$ x + jy$$
$$x + jy == Acos(\theta) + Asin(\theta)j$$
$$Acos(\theta) + Asin(\theta)j == A(cos(\theta) + sin(\theta)j) == Ae^{\theta j}$$
$$A = \sqrt{(x^2 + y^2)}$$
$$\theta == arctan(\frac y x)$$

In [ ]:
import numpy as np
from scipy.fftpack import fft, ifft
x = np.array([1.0, 2.0, 1.0, -1.0, 1.5])
y = fft(x)
print(y)
yinv = ifft(y)
print(yinv)
print(np.angle(y))
print(np.arctan(-1.65109876/2.08155948))

皮尔森相关系数（Pearson product-moment correlation coefficient）
$$\rho = \frac{\sum_{i=0}^{N-1} {(x_i - \mu_x)(y_i - \mu_y)}} {{N \sigma_x \sigma_y}}$$
$X$,$Y$是有$N$个测量值的向量，上式中的 $\mu_x$ 和 $\mu_y$ 是 $X$ 和 $Y$ 的均值， $\sigma_x$ 和 $\sigma_y$ 是它们的标准差。
$\rho$ 取值范围[-1,1] 

通常处理的信号为无偏（均值为0）归一化（标准差为1）的信号，所以：
$$\rho = \frac {1} {N} {\sum_{i=0}^{N-1} x_i y_i} $$
进一步：
$$r = {\sum_{i=0}^{N-1} x_i y_i} $$

如果把 $X$ 和 $Y$ 看做是向量，那么这个公式就是 点积（dot product） 的公式, 点积表征了信号之间的相似度
如果$X$ 和 $Y$ 都是归一化的：
$$X * Y = cos\theta$$

In [ ]:
import thinkdsp
import thinkplot

def make_sine(offset):
    signal = thinkdsp.SinSignal(freq=440, offset=offset)
    wave = signal.make_wave(duration=0.5, framerate=10000)
    return wave

wave1 = make_sine(offset = 0)
wave2 = make_sine(offset = np.pi / 2) # offset = 1

wave1.plot(label="offset 0")
wave2.plot(label="offset 1")

thinkplot.config(xlabel='Time (s)',
                 ylabel='Amplitude',
                 xlim=[0, 0.01])

In [ ]:
import numpy as np
corr_matrix = np.corrcoef(wave1.ys, wave2.ys)
print(corr_matrix)
wave1.corr(wave2)

In [ ]:
import matplotlib.pyplot as plt

#两个波形随着offset的不同，相关性发生的变化

wave_offsets = np.linspace(0, 2 * np.pi, 100)
corr_values = []
for i in range(len(wave_offsets)):
    wavei = make_sine(offset = wave_offsets[i])
    corr_values.append(wave1.corr(wavei))
    
plt.xlabel("offset") 
plt.ylabel("corrlation")
plt.plot(wave_offsets, corr_values)

In [ ]:
#序列相关性
def serial_corr(wave, lag=1):
    n = len(wave)
    y1 = wave.ys[lag:]
    y2 = wave.ys[:n-lag]
    corr = np.corrcoef(y1, y2)[0, 1]
    return corr

print(serial_corr(wave1, lag=0)) # 自相关
print(serial_corr(wave1, lag=6)) # T/4 自相关性最小
print(serial_corr(wave1, lag=11)) # 编译 T/2  lag = T/2 * framerate = 1/2f * frametate 10000/2*440 = 11.363636363636363
print(serial_corr(wave1, lag=23))

print(thinkdsp.__file__)

In [ ]:
#自相关 求基频
import os

thinkdsp_dir = os.path.dirname(thinkdsp.__file__)
wave = thinkdsp.read_wave(thinkdsp_dir + '/28042__bcjordan__voicedownbew.wav')
#wave.normalize()
wave.make_audio()


